In [5]:
from nonasymptotic.envs import StraightLine
from nonasymptotic.prm import SimplePRM

import numpy as np

In [6]:
seed = 1
prm_env_2d = StraightLine(dim=2, delta_clearance=0.5, seed=seed)
prm = SimplePRM(0.10, prm_env_2d.is_motion_valid, prm_env_2d.sample_from_env, prm_env_2d.distance_to_path, seed=seed)
prm.grow_to_n_samples(1000)

# plt.figure()
# for u, v in prm.g_prm.iterEdges():
#     coords_u = prm.samples[u]
#     coords_v = prm.samples[v]
# 
#     plt.plot([coords_u[0], coords_v[0]], [coords_u[1], coords_v[1]], 'ro-')
# 
# plt.show()

# some other computations to help other visual tests later on:
print('N Components: %i' % len(prm.g_cc.getComponents()))
print('N nodes: %i' % prm.num_vertices())
print('N edges: %i' % prm.num_edges())


N Components: 1
N nodes: 1000
N edges: 8212


In [7]:
import time

times = []
for _ in range(50):
    t_start = time.process_time()
    prm_env_2d.is_prm_epsilon_delta_complete(prm, 0.50, n_samples_per_check=100, timeout=60.0, timeout_area_tol=1e-6, vis=False)
    t_end = time.process_time()
    times.append(t_end - t_start)
    
print('mean: %f, std: %f' % (np.mean(times), np.std(times)))

covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
covered!
0.0004036820077975012


RuntimeError: Triangulation failed -- probably because of invalid geometry on input.

In [ ]:
import triangle as tr
import numpy as np
data = np.load('bad_triangulate.npz')
tri_out = tr.triangulate(data, 'p')